# Assignment 03

### Deadline: Next tutorial
### Deliverables: Quiz on this assignment in the next tutorial

In this assignment, you can use the same virtual environment used for the tutorial.

## 1. Symbolic Regression

### 1.1 ECSEL as symbolic regressor
First we look at ECSEL as symbolic regressor, thus where the ground truth equation is known, and we try to recover it exactly.

In [1]:
# import and settings

import numpy as np
import time
import warnings
from scipy.optimize import minimize

warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True, precision=4)

In [2]:
# helper functions

def create_term_sparse(const: float, var_powers: dict[int, float]):
    """Sparse term: const * Π x_var^power, where var indexes start at 1."""
    return {"const": float(const), "var_powers": dict(var_powers)}

def sparse_to_dense_terms(sparse_terms):
    """
    Converts sparse terms to dense terms with exponent vectors of length m.
    Returns: (dense_terms, m)
    """
    if len(sparse_terms) == 0:
        raise ValueError("No terms provided.")

    max_var = 0
    for term in sparse_terms:
        if term["var_powers"]:
            max_var = max(max_var, max(term["var_powers"].keys()))
    if max_var == 0:
        max_var = 1  # handle constant-only terms

    dense_terms = []
    for term in sparse_terms:
        exps = np.zeros(max_var, dtype=float)
        for var_idx, power in term["var_powers"].items():
            exps[var_idx - 1] = float(power)
        dense_terms.append({"const": float(term["const"]), "exponents": exps})

    return dense_terms, max_var

def evaluate_terms(X, dense_terms):
    """
    Evaluates each term on each sample.
    X: (n, m)
    dense_terms: list of {"const": c, "exponents": (m,)}
    returns term_values: (n, K)
    """
    X = np.asarray(X, dtype=float)
    n, m = X.shape
    K = len(dense_terms)
    term_vals = np.zeros((n, K), dtype=float)

    # keep strictly positive for non-integer exponents + log
    X_safe = np.clip(X, 1e-12, None)

    for k, term in enumerate(dense_terms):
        c = term["const"]
        e = term["exponents"]
        # Π x_j^e_j
        term_vals[:, k] = c * np.prod(X_safe ** e, axis=1)

    return term_vals

def generate_synthetic_data(n_samples, sparse_terms, x_bounds=(0.1, 5.0), noise_std=0.0, seed=42):
    """
    Generates y = sum_k c_k * Π x_j^{e_jk} + noise
    """
    rng = np.random.default_rng(seed)
    dense_terms, m = sparse_to_dense_terms(sparse_terms)

    X = rng.uniform(x_bounds[0], x_bounds[1], size=(n_samples, m))
    y = evaluate_terms(X, dense_terms).sum(axis=1)

    if noise_std > 0:
        y = y + rng.normal(0.0, noise_std, size=n_samples)

    return X, y, dense_terms

def params_to_terms(params, K, m):
    per = m + 1
    dense_terms = []
    for k in range(K):
        s = k * per
        c = params[s]
        e = params[s+1:s+1+m]
        dense_terms.append({"const": float(c), "exponents": np.array(e, dtype=float)})
    return dense_terms

def print_sparse_terms(sparse_terms, title="TRUE FUNCTION"):
    print(f"\n{title}:")
    for i, t in enumerate(sparse_terms, 1):
        c = t["const"]
        vp = t["var_powers"]
        if not vp:
            term = "1"
        else:
            term = " * ".join([f"x{v}^{p:g}" for v, p in sorted(vp.items())])
        sign = "+" if c >= 0 else ""
        print(f"  Term {i}: {sign}{c:g} * {term}")

def print_dense_terms(dense_terms, title="FITTED TERMS", max_terms=None):
    print(f"\n{title}:")
    shown = dense_terms if max_terms is None else dense_terms[:max_terms]
    for i, t in enumerate(shown, 1):
        c = t["const"]
        e = t["exponents"]
        parts = [f"x{j+1}^{e[j]:.3f}" for j in range(len(e)) if abs(e[j]) > 1e-6]
        term = " * ".join(parts) if parts else "1"
        sign = "+" if c >= 0 else ""
        print(f"  Term {i}: {sign}{c:.3f} * {term}")

def reconstruction_metrics(X, y, dense_terms):
    y_hat = evaluate_terms(X, dense_terms).sum(axis=1)
    mse = np.mean((y - y_hat) ** 2)
    rel = np.mean(np.abs(y - y_hat) / (np.abs(y) + 1e-12))
    return mse, rel, y_hat

In [3]:
# ECSEL

def unpack_params(params, K, m):
    """params -> list of (c_k, e_k) where e_k is length m"""
    params = np.asarray(params, dtype=float)
    per = m + 1
    terms = []
    for k in range(K):
        s = k * per
        c = params[s]
        e = params[s+1:s+1+m]
        terms.append((c, e))
    return terms

def objective_mse(params, X, y, K, m):
    try:
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)
        n = y.shape[0]
        X_safe = np.clip(X, 1e-12, None)

        y_pred = np.zeros(n, dtype=float)
        for c, e in unpack_params(params, K, m):
            y_pred += c * np.prod(X_safe ** e, axis=1)

        mse = np.mean((y - y_pred) ** 2)
        if not np.isfinite(mse):
            return 1e10
        return mse
    except Exception:
        return 1e10

def grad_mse(params, X, y, K, m):
    """
    Analytical gradient of MSE wrt constants + exponents.
    """
    try:
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)
        n = y.shape[0]
        X_safe = np.clip(X, 1e-12, None)
        logX = np.log(X_safe)

        per = m + 1
        grad = np.zeros_like(params, dtype=float)

        # forward
        term_vals = []
        y_pred = np.zeros(n, dtype=float)
        terms = unpack_params(params, K, m)
        for (c, e) in terms:
            t = c * np.prod(X_safe ** e, axis=1)
            term_vals.append(t)
            y_pred += t

        r = (y_pred - y)  # residual
        scale = 2.0 / n

        # backward
        for k, (c, e) in enumerate(terms):
            s = k * per
            base = np.prod(X_safe ** e, axis=1)  # without c

            # d/dc_k
            grad[s] = scale * np.sum(r * base)

            # d/de_kj
            # term = c * base; d(term)/de_j = term * log(x_j)
            term = c * base
            for j in range(m):
                grad[s + 1 + j] = scale * np.sum(r * term * logX[:, j])

        # guard
        grad = np.where(np.isfinite(grad), grad, 0.0)
        return grad
    except Exception:
        return np.ones_like(params, dtype=float) * 1e10
    
def fit_lbfgsb(X, y, K, n_restarts=10, bounds_const=(-10, 10), bounds_exp=(-5, 5), seed=42, use_grad=True):
    """
    Fits K-term signomial with L-BFGS-B and random restarts.
    params layout: [c1, e11..e1m, c2, e21..e2m, ...]
    """
    rng = np.random.default_rng(seed)
    n, m = X.shape
    per = m + 1
    n_params = K * per

    bounds = []
    for _ in range(K):
        bounds.append(bounds_const)
        bounds.extend([bounds_exp] * m)

    best = None
    best_val = np.inf

    print(f"L-BFGS-B: {n_restarts} restarts | K={K}, m={m}")
    for r in range(n_restarts):
        x0 = np.zeros(n_params, dtype=float)

        # heuristic init for restart 0
        if r == 0:
            for k in range(K):
                s = k * per
                x0[s] = rng.uniform(-1, 1)              # small const
                x0[s+1:s+1+m] = rng.uniform(0.5, 2.0, size=m)  # positive exponents
        else:
            for i, (lb, ub) in enumerate(bounds):
                x0[i] = rng.uniform(lb, ub)

        try:
            res = minimize(
                fun=lambda p: objective_mse(p, X, y, K, m),
                x0=x0,
                method="L-BFGS-B",
                jac=(lambda p: grad_mse(p, X, y, K, m)) if use_grad else None,
                bounds=bounds,
                options={"maxiter": 1000, "ftol": 1e-12}
            )
            if res.success and res.fun < best_val:
                best_val = res.fun
                best = res.x.copy()
                print(f"  restart {r+1}/{n_restarts}: improved -> MSE {best_val:.8e}")
        except Exception as e:
            print(f"  restart {r+1}/{n_restarts}: failed ({e})")

    return best, best_val

### Data generation
We apply symbolic regression on signomials of the AI Feynman database of equations (see related literature Udrescu & Tegmark, 2019). We choose equations I.12.1, I.12.2 and I.13.12 and generate 200 samples between 0.1 and 5.

In [4]:
# Data set

test_equations = {
    "I.12.1": [
        create_term_sparse(1.0, {1: 1.0, 2: 1.0, 3: 1.0}),
    ],
    "I.12.2": [
        create_term_sparse(1.0, {1: 1.0, 2: 1.0, 3: -2.0, 4: -1.0}),
    ],
    "I.13.12": [
        create_term_sparse(1.0, {1: 1.0, 2: 1.0, 3: 1.0, 5: -1.0}),
        create_term_sparse(-1.0, {1: 1.0, 2: 1.0, 3: 1.0, 4: -1.0}),
    ],
}

The following cell will run the experiments.

In [5]:
def run_experiments(
    equations: dict,
    n_samples=200,
    noise_std=0.01,
    n_restarts=12,
    x_bounds=(0.1, 5.0),
    seed=42
):
    for name, sparse_terms in equations.items():
        print("\n" + "=" * 80)
        print(f"FITTING EQUATION: {name}")
        start = time.time()

        K = len(sparse_terms)
        X, y, _ = generate_synthetic_data(
            n_samples=n_samples,
            sparse_terms=sparse_terms,
            x_bounds=x_bounds,
            noise_std=noise_std,
            seed=seed
        )
        n, m = X.shape

        print_sparse_terms(sparse_terms)
        print(f"\nData: n={n}, m={m} | y range [{y.min():.3f}, {y.max():.3f}]")

        fit_start = time.time()
        best_params, best_obj = fit_lbfgsb(
            X, y, K,
            n_restarts=n_restarts,
            seed=seed,
            use_grad=True
        )
        fit_time = time.time() - fit_start

        if best_params is None:
            print("Optimization failed.")
            continue

        fitted_terms = params_to_terms(best_params, K, m)
        print_dense_terms(fitted_terms, title="FITTED TERMS")

        mse, rel, _ = reconstruction_metrics(X, y, fitted_terms)
        print(f"\nMSE: {mse:.8e}")
        print(f"Mean relative error: {rel:.6f} ({rel*100:.3f}%)")
        print(f"Fit time: {fit_time:.2f}s | Total time: {time.time()-start:.2f}s")
        
        
np.random.seed(42)
run_experiments(test_equations, n_samples=200, noise_std=0.01, n_restarts=15)


FITTING EQUATION: I.12.1

TRUE FUNCTION:
  Term 1: +1 * x1^1 * x2^1 * x3^1

Data: n=200, m=3 | y range [0.099, 93.828]
L-BFGS-B: 15 restarts | K=1, m=3
  restart 1/15: improved -> MSE 1.10239351e-04
  restart 4/15: improved -> MSE 1.10239351e-04

FITTED TERMS:
  Term 1: +1.000 * x1^1.000 * x2^1.000 * x3^1.000

MSE: 1.10239351e-04
Mean relative error: 0.003775 (0.377%)
Fit time: 0.10s | Total time: 0.10s

FITTING EQUATION: I.12.2

TRUE FUNCTION:
  Term 1: +1 * x1^1 * x2^1 * x3^-2 * x4^-1

Data: n=200, m=4 | y range [-0.009, 422.965]
L-BFGS-B: 15 restarts | K=1, m=4
  restart 1/15: improved -> MSE 1.93462294e+03
  restart 2/15: improved -> MSE 9.95035846e-05
  restart 4/15: improved -> MSE 9.95035846e-05

FITTED TERMS:
  Term 1: +1.000 * x1^1.000 * x2^1.000 * x3^-2.000 * x4^-1.000

MSE: 9.95035846e-05
Mean relative error: 0.128648 (12.865%)
Fit time: 0.09s | Total time: 0.09s

FITTING EQUATION: I.13.12

TRUE FUNCTION:
  Term 1: +1 * x1^1 * x2^1 * x3^1 * x5^-1
  Term 2: -1 * x1^1 * x2^1 

#### Q1. What is the difference between equations I.12.1, I.12.2 and I.13.12?

#### Q2. Find an equation in the database (see AI Feynman paper) that is NOT recovered using ECSEL. Can you think of a reason why?

#### Q3. Scan the paper (ECSEL, Lumadjeng et al. (2026)). What would be the elasticity of equation I.12.1?

#### Q4. Can we read the elasticity as easy off of equations I.13.12? Explain your answer.


### 1.2 ECSEL as classifier

We use ECSEL on the Online Shopping Intention dataset. This is a dataset of ~12K samples/users that browser an online shopping website. The task is to predict whether a user session will end in a purchase or not. The equations found by ECSEL is given by

<div>
<img src="../img/osi_ecsel_signomial.png" width="350"/>
</div>

where the sigmoid threshold is equal to p=0.559

Q5. Explain what this threshold means

Q6. Looking at the equation, what can you say about how predictions are made


Q7. Give a global explanation using an elasticity plot. Explain what we see here.

In [ ]:
# Code Q7

Q8. Give a local explanation via a waterfall plot. Choose an instance yourself. Explain what we see here.

In [6]:
# Code Q8

## 2. Counterfactual Explanations Using Optimization with Constraint Learning (CE-OCL)

Read the paper (main text is 6 pages). You will be quizzed on comprehension, methodology and interpretability of results. You can find the paper in the directory, or go to Canvas > Modules > Related Literature of this week.